## Tracking Models Available in the OpenCV Tracker Class
1. Boosting
2. CSRT
3. KCF
4. MEDIANPLOW
5. MIL
6. MOSSE
7. TLD

Điểm nổi bật ngắn gọn của mỗi Trình theo dõi đối tượng bên dưới:
Trình theo dõi tăng cường: Dựa trên cùng một thuật toán được sử dụng để cung cấp năng lượng cho máy học đằng sau Haar Cascades (Adaboost), nhưng giống như Haar Cascades, đã hơn một thập kỷ. Trình theo dõi này chậm và không hoạt động rất tốt. Thú vị chỉ vì lý do di sản và so sánh các thuật toán khác.

Trình theo dõi MIL: Độ chính xác tốt hơn so với việc tăng cường theo dõi nhưng thực hiện một công việc kém về báo cáo thất bại.

Trình theo dõi KCF: Bộ lọc tương quan hạt nhân. Nhanh hơn tăng và mil. Tương tự như MIL và KCF, không xử lý tốt việc tắc hoàn toàn. Bộ lọc Ation (với độ tin cậy của kênh và không gian). Có xu hướng chính xác hơn KCF nhưng chậm hơn một chút.

Trình theo dõi CSRT: Bộ lọc tương quan phân biệt đối xử (với độ tin cậy của kênh và không gian). Có xu hướng chính xác hơn KCF nhưng chậm hơn một chút.

Trình theo dõi MedianFlow: Làm một công việc tốt báo cáo thất bại; Tuy nhiên, nếu có quá nhiều bước nhảy trong chuyển động, chẳng hạn như các đối tượng di chuyển nhanh hoặc các đối tượng thay đổi nhanh chóng về ngoại hình của chúng, mô hình sẽ thất bại.

Trình theo dõi TLD: Tôi không chắc chắn có vấn đề với việc triển khai OpenCV của Trình theo dõi TLD hay chính thuật toán thực tế không, nhưng Trình theo dõi TLD cực kỳ dễ bị ảnh hưởng đến các chất giả. Tôi không khuyên bạn nên sử dụng trình theo dõi đối tượng OpenCV này.

Trình theo dõi Mosse: Rất, rất nhanh. Không chính xác như CSRT hoặc KCF nhưng là một lựa chọn tốt nếu bạn cần tốc độ thuần túy.

Goturn Tracker: Trình phát hiện đối tượng dựa trên học tập sâu duy nhất có trong OpenCV. Nó yêu cầu các tệp mô hình caffe bổ sung để chạy.



Sử dụng CSRT khi bạn cần độ chính xác theo dõi đối tượng cao hơn và có thể chịu được thông lượng FPS chậm hơn
Sử dụng KCF khi bạn cần thông lượng FPS nhanh hơn nhưng có thể xử lý độ chính xác theo dõi đối tượng thấp hơn một chút
Sử dụng mosse khi bạn cần tốc độ tinh khiết

In [ ]:
import cv2
import sys
import os
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip

In [ ]:
input_video = '/content/5991156-hd_1280_720_30fps.mp4'
clip = VideoFileClip(input_video)
clip.ipython_display(width=640)

## 2. Define Annotation Convenience Functions

In [ ]:
def drawBannerText(frame, text, banner_height_percent=0.08, font_scale=0.8, text_color=(0, 255, 0), font_thickness=2):
    """
    Vẽ một banner đen trên đầu khung hình và thêm văn bản vào đó.
    """
    # Xác định chiều cao banner
    banner_height = int(banner_height_percent * frame.shape[0])

    # Vẽ hình chữ nhật đen làm banner
    cv2.rectangle(frame, (0, 0), (frame.shape[1], banner_height), (0, 0, 0), thickness=-1)

    # Thêm văn bản vào banner
    left_offset = 20
    location = (left_offset, int(15 + (banner_height_percent * frame.shape[0]) / 2))
    cv2.putText(frame, text, location, cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_color, font_thickness, cv2.LINE_AA)

def drawRectangle(frame, bbox, color=(255, 0, 0)):
    """
    Vẽ một hình chữ nhật trên khung hình theo tọa độ bbox.
    """
    p1 = (int(bbox[0]), int(bbox[1]))
    p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
    cv2.rectangle(frame, p1, p2, color, 2, 1)

def displayRectangle(frame, bbox, color=(255, 0, 0)):
    """
    Hiển thị khung hình với hình chữ nhật vẽ trên đó.
    """
    plt.figure(figsize=(20, 10))
    frameCopy = frame.copy()
    drawRectangle(frameCopy, bbox, color)
    frameCopy = cv2.cvtColor(frameCopy, cv2.COLOR_RGB2BGR)
    plt.imshow(frameCopy)
    plt.axis('off')
    plt.show()

def drawText(frame, text, location=(20, 20), font_scale=1, color=(50, 170, 50), font_thickness=2):
    """
    Vẽ văn bản lên khung hình tại vị trí chỉ định.
    """
    color = (0, 255, 0)
    font_scale = 0.8
    cv2.putText(frame, text, location, cv2.FONT_HERSHEY_SIMPLEX, font_scale, color, font_thickness, cv2.LINE_AA)



## 3. Create a Tracker Instance

In [ ]:
# Set up tracker.
tracker_types = ['BOOSTING', 'CSRT', 'KCF', 'MEDIANFLOW', 'MIL', 'MOSSE', 'TLD']

# Change the index to change the tracker type.
tracker_type = tracker_types[0]

# Create the selected tracker
if tracker_type == 'BOOSTING':
    tracker = cv2.legacy.TrackerBoosting_create()
elif tracker_type == 'CSRT':
    tracker = cv2.legacy.TrackerCSRT_create()
elif tracker_type == 'KCF':
    tracker = cv2.legacy.TrackerKCF_create()
elif tracker_type == 'MEDIANFLOW':
    tracker = cv2.legacy.TrackerMedianFlow_create()
elif tracker_type == 'MIL':
    tracker = cv2.legacy.TrackerMIL_create()
elif tracker_type == 'MOSSE':
    tracker = cv2.legacy.TrackerMOSSE_create()
elif tracker_type == 'TLD':
    tracker = cv2.legacy.TrackerTLD_create()

## 4. Create Video Capture and Video Writer Objects

In [ ]:
# Read video
video_input_file_name = "/content/5991156-hd_1280_720_30fps.mp4"

# Create output file name for annotated video
video_output_file_name = os.path.splitext(os.path.basename(video_input_file_name))[0] + "_" + tracker_type + ".mp4"

# Create video capture object
video_cap = cv2.VideoCapture(video_input_file_name)

# Read first frame from video
ok, frame = video_cap.read()

# Confirm video file can be opened
if video_cap.isOpened():
    width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video_cap.get(cv2.CAP_PROP_FPS))
else:
    print("Could not open video")
    sys.exit()

fps_write = 20  # Slow down video for demonstration purposes

# Set up video writer for mp4
fourcc = cv2.VideoWriter_fourcc(*"mp4v")

# Create video writer object
video_out = cv2.VideoWriter(video_output_file_name, fourcc, fps_write, (width, height))

## 5. Define the Initial BBox

In [ ]:
# Define a bounding box for the location of the object in the first video frame.
bbox = (530, 310, 420, 180)  # Coordinates for race car

# Check if a bounding box has been defined. If not, allow the user to select it manually.
if bbox is None:
    # Display the first frame of the video and use the mouse to specify a bounding box.
    # When done, press the space bar or Enter key to confirm.
    bbox = cv2.selectROI(frame, False)  # Note: May cause Python to hang on MacOS.
    print(bbox)

# Create a copy of the frame and display the bounding box.
frame_copy = frame.copy()
displayRectangle(frame_copy, bbox, color=(0, 255, 255))

## 6. Intilialize Trạcker

In [ ]:
tracker.init(frame, bbox)

## 7. Processes Video Frame and Track Object

In [ ]:
while True:
    ok, frame = video_cap.read()
    if not ok:
        break

    # Start timer
    timer = cv2.getTickCount()

    # Update tracker
    ok, bbox = tracker.update(frame)

    # Calculate Frames per second (FPS)
    fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer)

    # Draw bounding box
    if ok:
        drawRectangle(frame, bbox, color=(0, 255, 255))
    else:
        drawText(frame, "Tracking failure detected", location=(80, 140), color=(0, 0, 255))

    # Display Info
    drawBannerText(frame, f"{tracker_type} Tracker, FPS: {int(fps)}")

    # Write frame to video
    video_out.write(frame)

# Release video capture and writer objects
video_cap.release()
video_out.release()

In [ ]:
from moviepy.editor import *
clip = VideoFileClip(video_output_file_name)
clip = clip.resize(height=600)
clip.ipython_display()